# Pluto Plotting

of the Kelvin-Helmholtz instability

To run this notebook, you need to download and run the `PLUTO` code. The setup files, instructions and this noteook can be downloaded from [github](https://github.com/birnstiel/pluto_khi_setup). Instructions can be found in the repositories [`Readme.md`](Readme.md).

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import animation
import numpy as np
import glob
import pyPLUTO as pp

Read in the data

In [ ]:
files = glob.glob('data*.dbl')
D = [pp.pload.pload(i, w_dir='./') for i in range(len(files))]

Set some plotting parameters

In [ ]:
q = len(D[0].x1)//40; # for quiver plots: plot every qth arrow only
quantity = 'tr1' # 'rho', 'tr1'

In [ ]:
if quantity == 'rho':
    name = 'Density'
    vmin = [getattr(_D,quantity).min() for _D in D][-1]
    vmax = [getattr(_D,quantity).max() for _D in D][-1]
elif quantity == 'tr1':
    vmin = 0
    vmax = 1
    name = 'Tracer'

## Static plotting

In [ ]:
D1 = D[-1]

f,ax = plt.subplots(figsize=(10, 4))
cc = ax.pcolormesh(D1.x1r, D1.x2r, D1.tr1.T, vmin=0, vmax=1)
ax.set_title(f'{name}, time = {D1.SimTime:.2f}')
ax.set_aspect(1)
ax.set_xlabel('x')
ax.set_ylabel('y')
qq=ax.quiver(D1.x1[::q], D1.x2[::q], D1.vx1[::q,::q].T, D1.vx2[::q,::q].T, scale=None)
cc=f.colorbar(cc, ax=ax)
f.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.95);

## Create an animation

In [ ]:
def animate(i,init=False):
    if init:
       
        cc = ax.pcolormesh(D[0].x1r, D[0].x2r, getattr(D[0], quantity).T, edgecolors='face',
                           lw=0.5, alpha=1.0, vmin=vmin, vmax=vmax)
        qq = ax.quiver(D[0].x1[::q], D[0].x2[::q], D[i].vx1[::q, ::q].T, D[i].vx2[::q, ::q].T, scale=None)
        ax.set_aspect(1)
        ax.set_xlabel('$x$')
        ax.set_ylabel('$y$')
        cb=plt.colorbar(cc, ax=ax)
        ax.set_title(f'{name}, time = {D[i].SimTime:.2f}')
        f.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.95);
        
        return cc
    
    else:
        quad1 = ax.findobj(lambda o: type(o)==mpl.collections.QuadMesh)[0]
        quad1.set_array(getattr(D[i], quantity).T.ravel())
        
        quiver1 = ax.findobj(lambda o: type(o)==mpl.quiver.Quiver)[0]
        quiver1.set_UVC(D[i].vx1[::q, ::q].T, D[i].vx2[::q, ::q].T)
        
        ax.set_title(f'{name}, time = {D[i].SimTime:.2f}')

        return quad1, quiver1
    
mpl.rc('animation', html='html5')
f,ax = plt.subplots(1, 1, figsize=(10, 4))
anim = animation.FuncAnimation(f, animate, frames=len(files),
                               init_func=lambda: animate(0, init=True),
                               interval=33, blit=False)
plt.close(f)

In [ ]:
display(anim)